# Web Scraping of Machine Learning Mastery Blog Entries Using Python and BeautifulSoup
### David Lowe
### January 6, 2019

SUMMARY: The purpose of this project is to practice web scraping by extracting specific pieces of information from a website. The web scraping code was written in Python 3 and leverages the BeautifulSoup module.

INTRODUCTION: Dr. Jason Brownlee’s Machine Learning Mastery hosts its tutorial lessons at https://machinelearningmastery.com/blog. The purpose of this exercise is to practice web scraping using Scrapy by gathering the blog entries from Machine Learning Mastery’s web pages. This iteration of the script automatically traverses the web pages to capture all blog entries and store all captured information in a JSON output file.

Starting URLs: https://machinelearningmastery.com/blog

## Loading Libraries and Packages

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import smtplib
from email.message import EmailMessage
from datetime import datetime
import urllib.request
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
from random import randint
from time import sleep

startTimeScript = datetime.now()

## Setting up the email notification function

In [2]:
def email_notify(msg_text):
    sender = "luozhi2488@gmail.com"
    receiver = "dave@contactdavidlowe.com"
    with open('../email_credential.txt') as f:
        password = f.readline()
        f.close()
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Python Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender, password)
    server.send_message(msg)
    server.quit()

In [3]:
email_notify("The web scraping process has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

## Setting up the necessary parameters

In [4]:
# Specifying the URL of desired web page to be scrapped
starting_url = "https://machinelearningmastery.com/blog"
# website_url = "https://papers.nips.cc"

# Creating an html document from the URL
uastring = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.80 Safari/537.36"
req = urllib.request.Request(
    starting_url,
    data=None,
    headers={'User-Agent': uastring}
)

try:
    session = urllib.request.urlopen(req)
except HTTPError as e:
    print('The server could not serve up the page!')
    print(e)
    sys.exit(1)
except URLError as e:
    print('The server could not be reached!')
    print(e)
    sys.exit(1)

try:
    webpage = BeautifulSoup(session.read(), 'html5lib')
except AttributeError as e:
    print('Page title could not be found - Might indicate problems!')
    sys.exit(1)
else:
    print('Successfully accessed the web page: ' + starting_url)

Successfully accessed the web page: https://machinelearningmastery.com/blog


## Performing the Scraping and Processing

In [5]:
email_notify("The web page loading and item extraction process has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Setting up a dataframe to capture the records
df = pd.DataFrame(columns=['blog_title','blog_url','date','author','summary'])
i = 0

article_list = webpage.find_all('article')

done = False

while not done :
    for article_item in article_list:

        blog_title = "[Not Found]"
        blog_url = "[Not Found]"
        date = "[Not Found]"
        author = "[Not Found]"
        summary = "[Not Found]"

        blog_title = article_item.header.h2.string
        blog_url = article_item.a["href"]
        date = article_item.abbr.string
        author = article_item.find(class_="fn").a.string
        summary = article_item.section.p.string
        df.loc[i] = [blog_title,blog_url,date,author,summary]
        i = i + 1
#         print(blog_title,blog_url,date,author,summary)

    next_page_css = webpage.find(class_="next page-numbers")

    if next_page_css != None :
        next_page_url = next_page_css["href"]

        # Adding random wait time so we do not hammer the website needlessly
        waitTime = randint(3,5)
        sleep(waitTime)
        print("Waited " + str(waitTime) + " seconds to retrieve the next URL.")
        req = urllib.request.Request(
            next_page_url,
            data=None,
            headers={'User-Agent': uastring}
        )

        try:
            session = urllib.request.urlopen(req)
        except HTTPError as e:
            print('The server could not serve up the page!')
            print(e)
            sys.exit(1)
        except URLError as e:
            print('The server could not be reached!')
            print(e)
            sys.exit(1)

        try:
            webpage = BeautifulSoup(session.read(), 'html5lib')
        except AttributeError as e:
            print('Page title could not be found - Might indicate problems!')
            sys.exit(1)
        else:
            print('Successfully accessed the web page: ' + next_page_url)
            article_list = webpage.find_all('article')
    else :
        done = True

Waited 5 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/2/
Waited 4 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/3/
Waited 4 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/4/
Waited 5 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/5/
Waited 5 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/6/
Waited 5 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/7/
Waited 5 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/8/
Waited 3 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.c

Successfully accessed the web page: https://machinelearningmastery.com/blog/page/66/
Waited 4 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/67/
Waited 3 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/68/
Waited 5 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/69/
Waited 3 seconds to retrieve the next URL.
Successfully accessed the web page: https://machinelearningmastery.com/blog/page/70/


## Organizing Data and Producing Outputs

In [6]:
out_file = df.to_json(orient='records')
with open('web-scraping-py-bsoup-nips-proceedings.json', 'w') as f:
    f.write(out_file)
email_notify("The web scraping process has completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:06:06.208846
